# Introduction to ConverSight Context Manager

The context manager in ConverSight is an module that defines the common functionalities used by tasks & flows. The primary purpose of a context manager is to simplify the management of resources, such as maintaining session attributes within the process and storing large data in the object store and get it back later.

Let's import the context manager from conversight library

In [1]:
from conversight import Context

Since this context is internally managed by Athena, all you have to do is pass this Context as an argument to the conversight tasks. This Context module has the following features built in 
- set => This method stores the data in the object store with reference to the given key
  > - key: It is the name that attaches the reference to the value
  > - value: Actual value to be stored, this could be anything
- get => This method gets the data from the objecct store based on the given key
  > - key: It's the reference name to retrieve the stored data .
- session => This object holds the most essential session attributes about the running task/flow
- waitUntilDatasetActive => This method checks the given dataset(s) are active or not and put a pause on the flow until the dataset(s) becomes active otherwise raise an exception and stop the flow
  > - datasetIds: List of dataset Ids to be verified
  > - maxRetries: Number of times to try if  dataset is not active, the default value is 2
  > - maxWaitTime: Number of seconds to wait before next try, the default value is 10
  > - incrementMultiplier: Number of times to increase the maxWaitTime, default value is 2 (maxWaitTime * incrementCount => 10 * 2 => 20 seconds) 

Let's create a simple task with context manager as an argument

In [2]:
from conversight import task

In [6]:
@task(name="context_example", tags=["context", "calculation"])
def cs_context(x: str, ctx: Context) -> int:
    """A simple context function"""
    try:
        return f"{x} {ctx.session.display_name} I got your name from the session"
    except Exception as e:
        return e

In [7]:
cs_context.run("Hello")

'Hello Athena I got your name from the session'

Let's create couple of tasks to see the set and get features using a flow

In [35]:
@task(name="context_set", tags=["context", "setter"])
def context_set(set_key: str, set_value: Any, ctx: Context) -> str:
    """A simple context setter function"""
    try:
        response = ctx.set(key=set_key, value=set_value)
        ctx.log.info(f"Value stored in {set_key}")
        return set_key
    except Exception as e:
        return e

In [36]:
context_set.register("Context_Examples", "A simple context setter function", "edit", True, True)

[2023-11-29 19:39:20,870] [INFO] context_set has been successfully registered. The most recent version available is 0.2 !!


In [19]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randint(0,10, size=(1,8)))
df

,0,1,2,3,4,5,6,7
0,3,9,9,1,0,9,5,1


In [10]:
from typing import Any

In [40]:
@task(name="context_get", tags=["context", "getter"])
def context_get(get_key: str, ctx: Context) -> Any:
    """A simple context getter function"""
    try:
        response = ctx.get(key=get_key)
        ctx.log.info(f"Value retrieved from {get_key}")
        return response
    except Exception as e:
        return e

In [41]:
context_get.register("Context_Examples", "A simple context getter function", "edit", True, True)

[2023-11-29 19:45:16,912] [INFO] context_get has been successfully registered. The most recent version available is 0.2 !!


In [29]:
from conversight import Flow, Parameter, TaskLibrary

In [42]:
tsk = TaskLibrary()

Tasks loaded  !!


In [47]:
description="A simple context manager flow"
with Flow(name="Getter_Setter", description=description, tags = ["Context", "getter", "setter"]) as flow:
    x = Parameter('set_key', "my_key")
    y = Parameter('set_value', "my_key")
    z = Parameter('get_key', "")
    
    xx = tsk.Context_Examples.context_set(x, y)
    zz = tsk.Context_Examples.context_get(xx)

In [48]:
flow.run(set_key="my_key", get_key="my_key", set_value=df)

skipLog is: False
[2023-11-29 20:05:34,862] [INFO] [Main-Flow]  Received request from AI Workbench, considering as test run..
[2023-11-29 20:05:34,865] [INFO] Context Actor [8386939bdff341ee822ae74db94da72d] deployed..
(runWrapperNotebook pid=664) [2023-11-29 20:05:50,855] [INFO] Value stored in my_key
[2023-11-29 20:05:51,837] [INFO] [Main-Flow]  Flow response: [   0  1  2  3  4  5  6  7
0  3  9  9  1  0  9  5  1]


[   0  1  2  3  4  5  6  7
 0  3  9  9  1  0  9  5  1]

(runWrapperNotebook pid=683) [2023-11-29 20:05:51,834] [INFO] Value retrieved from my_key


In [49]:
flow.register(libraryName="Context_Examples", flowName="Getter_Setter", description=description)

'The flow Getter_Setter has been registered successfully. Latest version available now is 0.1 !!'

In [50]:
ctx.waitUntilDatasetActive(["655ceb56-HrXES9SSm"])

[2023-11-29 20:19:37,622] [INFO] Dataset [655ceb56-HrXES9SSm] is active
[2023-11-29 20:19:37,622] [INFO]  All datasets are verified and in Active status !!


{'status': 'success',
 'message': 'All datasets are verified and in Active status'}